### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Uses REGEX to extract information from text
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/ If you run out of space in your linux directory the script will crash. There is a cache file written by selenium that may then need to be deleted to get the chrome driver working again. Follow the path in the error message and delete that cache directory

In [ ]:
pip install "numpy<2.0" 

In [7]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime
from nameparser import HumanName

### List of URLs input to search short list manually entered

In [ ]:
urls = ['https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006523?emailalert=true',
       'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006528?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006510?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006453?emailalert=true'
       ]

### Find URLS from saved email in html - save as from outlook in htm format

In [6]:
input = (r'IJSEMemail29.htm')
output = (r'NameCheckweek29a.xlsx')

In [8]:
#with open ('IJSEMemail1.htm', encoding = 'unicode_escape') as f:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')  
#print(soup.prettify())

In [9]:
urls = []
numberoflinks = 0
for url in soup.findAll( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
    print(url)
print(numberoflinks)
#print(urls)
#remove_list = ['TandC','doi.org', 'myaccount']
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e and "join-the-society" not in e]

#print(dois)
#print(urls)

<a alt="Discover publishing discounts and more by becoming a member" href="https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fmicrobiologysociety.org%2Fmembership%2Fjoin-the-society.html%3Futm_source%3DETOC%26utm_medium%3DBanner%26utm_campaign%3DMembership&amp;data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018527510%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&amp;sdata=9cAiYToxxTYjt8w%2FgemfeKqTjUbkLDkU5fso9q5TjSE%3D&amp;reserved=0"><span style="text-decoration:none;text-underline:none"><img border="0" class="fr-fic fr-dii" id="_x0000_i1026" src="https://www.microbiologyresearch.org/upload/marketing/ETOC-MemberBenefits-Banner-resized.png" style="max-width:100%" title="Member benefits"/></span></a>
<a href="https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%

### Main body - Selenium to extract data from html
### https://nameparser.readthedocs.io/en/latest/index.html to find the last name

In [10]:
# Remove non-ASCII characters
def remove_non_ascii(text):
    return ''.join(char for char in text if ord(char) < 128)


pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'Authority', 'DOI', 'URL'])
pd.set_option('display.max_columns', None)
combined_description = []
for url in urls:
    counter = 1
    strains = []
    accessions = []
    doi = []
    author = []
    date = []
    year = []
    author_raw = []
    author = []
    name1 = []
    name2 = []
    author_count = []
    authority = []
    description = None
    snumber = []    
    #scrape_with_selenium(url, counter)
    #scrape_with_beautifulsoup(url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)
    (driver.page_source).encode('utf-8')
    #(driver.page_source).encode('ascii', 'ignore')

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        #print(doi)
    
    #find publication year
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[3]/span/span[2]"):
        date = element.text
        year = date[-4:]
        #print(year)
    
    #find authors
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[1]/span"):
        author_raw = element.text
        #print(author_raw)
        author = re.sub(r"[\d+]+",'',author_raw)
        author = re.sub(r"†",'',author)
        author = re.sub(r" and ",',',author)
        author = re.sub(r",,",',',author)  
        author = author.split(',')
        author[:] = [item for item in author if item != '']
        #print(author)
        author_count = len(author)
        #print(author_count)
        if author_count == 1:
            name1 = (str(author[0])) 
            name1 = HumanName(name1)
            authority = name1.last + ' ' +str(year)
            #print(authority)
        elif author_count ==2:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            name2 = (str(author[1]))
            name2 = HumanName(name2)
            authority = name1.last + ' and ' + name2.last + ' ' +str(year)
            #print(authority)
        else:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            authority = name1.last + ' et al. ' +str(year)
            #print(authority)
    
    #extract data from the species description
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description: 
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                description = description.replace('\u2009', ' ')
                cleaned_text = remove_non_ascii(description)
                print(cleaned_text)
                combined_description.append(cleaned_text)
        
                #find the organism names    
                match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                orgname = [m.group() for m in regex.finditer(cleaned_text)]
                print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(cleaned_text)]
                    print('accessions', accessions)
    
                #find the strains
                #strainpattern = [r'(?<=type strain).*'] 
                strainpattern = [r'(?<=type strain).*?(?=\))']
                regex = re.compile(r'\b(' + '|'.join(strainpattern) + r')\b')
                if description is not None:
                    strains = [m.group() for m in regex.finditer(cleaned_text)]
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, authority, doi, url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK')
    
#Close the browser window
    driver.quit()    



Roseomonas cutis sp. nov., isolated from human epidermis
Description of Roseomonas cutis sp. nov.
 Roseomonas cutis (cutis. L. gen. n. cutis, of the skin)
Cells are Gram-stain-negative, pink pigmented, strictly aerobic, motile and rod shaped. Cell sizes are 1.41.60.60.7 m. Can assimilate sodium pyruvate, d-fructose, sucrose, d-mannose, disodium succinate, dl-xylose, dl-arabinose, N-acetylglucosamine, maltose, sodium gluconate, sodium maleate and d-galacturonic acid. But, cannot assimilate d-glucose, glycerol, lactose, d-sorbitol, d-mannitol, d-galactose, d-rhamnose, sodium acetate and trisodium citrate. Oxidase and catalase are positive. Growth occurs at 740 C (optimum, 28 C) and pH 6.09.5 (optimum, pH 7.0), and cells tolerate up to 3% NaCl; positive for urease, starch hydrolysis, carboxymethyl cellulose hydrolysis and DNase activities; negative for haemolysis, methyl red test, coagulase, gelatin, Tween 20, Tween 80 and casein hydrolysis activities. Nitrate is reduced to nitrite. In AP

Pseudomonas fungipugnans sp. nov., a potently antifungal bacterium isolated from moss
Description of Pseudomonas fungipugnans sp. nov.
 Pseudomonas fungipugnans (fun.gi.pugnans. L. masc. n. fungus, mushroom; L. pres. part. pugnans, fighting; N.L. part. adj. fungipugnans, fighting fungi).
P. fungipugnans cells are Gram-negative, motile rods (mean length of 2.60 m, mean width of 0.67 m) with one to three polar flagella. The organism is aerobic and gives round, beige mucoid colonies on LB agar at 30 C. It also grows on trypticase soy and M9 salt minimal agar. Optimal growth is at 30 C; however, growth also occurs at low temperatures (8 C). Growth is strong in the presence of 03% NaCl and weak at 4% NaCl, with optimal growth at 1% NaCl. Maximal growth occurs at pH 67, and weak growth at pH 8. Colonies show fluorescence on Kings medium B and produce siderophores on CAS medium. P. fungipugnans is oxidase positive. Cells hydrolyse arginine under anaerobic conditions and produce a hypersensiti

Four novel nontuberculous mycobacteria species: Mycobacterium wendilense sp. nov., Mycobacterium burgundiense sp. nov., Mycobacterium kokjensenii sp. nov. and Mycobacterium holstebronense sp. nov. revived from a historical Danish strain collection
Tistrella arctica sp. nov., isolated from the surface seawater of the Bering Sea and Chukchi Sea and emended description of the genus Tistrella
Luteimonas salinilitoris sp. nov., isolated from the shore soil of saline lake in Tibet of China
Description of Luteimonas salinilitoris sp. nov
 Luteimonas salinilitoris (sa.li.ni.li'to.ris. L. masc. adj. salinus, salty; L. neut. n. litor, shore, coast; N.L. gen. n. salinilitoris, of or coming from a salty shore)
Cells are Gram-stain-negative, aerobic, 0.500.53 m wide and 2.543.96 m long. Colonies are yellow, circular and convex after being cultivated on MA at 30  for 72 h. Growth is observed at temperatures ranging from 4 to 35 (optimum,30), in the presence of 08.0% (w/v) NaCl (optimum, 6.0%) and at

Genome-based reclassification of Suonthocola fibrivorans as a later heterotypic synonym of Diplocloster agilis
Vishniacozyma siamensis sp. nov., a new anamorphic tremellomycetous yeast species isolated from a mangrove forest in Thailand
Description of Vishniacozyma siamensis Gungprakhon, Khammeankae, Limtong and Khunnamwong sp. nov
 Vishniacozyma siamensis (si.am.ensis. N.L. fem. adj. siamensis, of or belonging to Siam, the old name of Thailand, where the type strain was isolated).
After 7 days at 25 C on YM agar, the colony is cream-coloured, convex and has an entire margin (Fig. 2a, b). The yeast cells are subglobose to ovoid (1.21.8 m1.62.9 m). Multilateral budding is observed (Fig. 2c). Pseudohyphae are not formed in slide culture on PDA after 7 days at 25 C. Ballistospore formation is not observed on YM agar, 5% malt extract agar and corn meal agar after 14 days at 25 C. Sexual reproduction is not observed on PDA, corn meal agar, 5% malt extract agar, YPD agar, YM agar, Fowells ac

In [ ]:
#write description to a file
#print(combined_description)
file = open('all_descriptions29', "w")
file.writelines(combined_description)
file.close()

In [11]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [','.join(map(str, l)) for l in pub_df['Strains']]
#pub_df

In [12]:
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
#pub_df

In [14]:
#try drop duplicate accessions here
pub_df = pub_df.drop_duplicates(subset='Accessions', keep="first")
pub_df.explode(['PublishedName']).reset_index(drop=True)
#pub_df

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,Roseomonas cutis,[OP788068],"is OT10T (=KCTC 92087T =JCM 34968T, is 72.6 mol% (genome",Son et al. 2025,https://doi.org/10.1099/ijsem.0.006617,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006617%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018546452%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=qPgbRuHApRO85zY1%2FXyHK7fwpRhZIQDtoO%2BMyTwMQ8I%3D&reserved=0
1,and Baghela,[MB855459],,Sipiczki and Baghela 2025,https://doi.org/10.1099/ijsem.0.006629,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006629%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018597448%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=gTvoT9taoY5EywTBMznivistMD3RQuxyiE0TB92Vbu8%3D&reserved=0
2,Methylobacterium litchii,"[PP600109, JBBXMY000000000]",is SD21T (=GDMCC 1.4327T=KCTC 8300T,Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
3,Methylobacterium guangdongense,"[PP600110, JBBXMX000000000]",is SI9T (=GDMCC 1.14329T= KCTC 8298T,Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
4,Methylorubrum subtropicum,"[PP600111, JBBXMW000000000]",is SB2T (=GDMCC 1.14328T= KCTC 8299T,Gao et al. 2025,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
5,Pseudomonas fungipugnans,"[OQ547267, JARBWL000000000]",,Schnyder et al. 2025,https://doi.org/10.1099/ijsem.0.006624,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006624%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018675073%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=%2FgfJBV%2BzhTm8J8IFPtJLLRJT15dKr4qhPOWJ5ISjMQY%3D&reserved=0
6,P. fungipugnans,"[OQ547267, JARBWL000000000]",,Schnyder et al. 202

In [15]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
#pub2_df

In [16]:
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
#pub4_df

In [17]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique = df_unique.dropna()
#df_unique

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [18]:
df_unique['accession'] = df_unique['accession'].astype('str') 

In [19]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [20]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


Error: Malformatted ID "nan"


0

In [21]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna()
srcchk_df 
#many need to add dropna here
#If /strain is not populated on the NCBI accession, we lose the NCBI srcchk info here

,accession,NCBIname,taxid,strain
0,OP788068,Roseomonas sp. OT10,2897332.0,OT10
2,PP600109,Methylobacterium sp.,409.0,SD21
3,JBBXMY000000000,Methylobacterium sp. SD21,3138810.0,SD21
4,PP600110,Methylobacterium sp.,409.0,SI9
5,JBBXMX000000000,Methylobacterium sp. SI9,3138811.0,SI9
6,PP600111,Methylorubrum sp.,2282524.0,SB2
7,JBBXMW000000000,Methylorubrum sp. SB2,3138812.0,SB2
8,OQ547267,Pseudomonas sp.,306.0,#681
9,JARBWL000000000,Pseudomonas sp. 681,3024217.0,681
11,PQ066384,Leptospira sp. WS58.C1,3235173.0,WS58.C1


### Combine dataframes into one

In [22]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'Authority', 'taxid', 'DOI', 'URL' ]]
#combine_df

In [23]:
def highlight_rows(row):
    ijsemvalue = row.loc['PublishedName']
    ncbivalue = row.loc['NCBIname']
    if ijsemvalue != ncbivalue:
        color = '#FFB3BA' # Red
    elif ijsemvalue == ncbivalue:
        color = '#BAFFC9' # Green
    return ['background-color: {}'.format(color) for r in row]

new_df = combine_df.style.apply(highlight_rows, axis=1, subset=['PublishedName', 'NCBIname'])
new_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,Roseomonas cutis,Roseomonas sp. OT10,"is OT10T (=KCTC 92087T =JCM 34968T, is 72.6 mol% (genome",OP788068,OT10,Son et al. 2025,2897332.000000,https://doi.org/10.1099/ijsem.0.006617,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006617%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018546452%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=qPgbRuHApRO85zY1%2FXyHK7fwpRhZIQDtoO%2BMyTwMQ8I%3D&reserved=0
1,and Baghela,nan,,MB855459,nan,Sipiczki and Baghela 2025,nan,https://doi.org/10.1099/ijsem.0.006629,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006629%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018597448%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=gTvoT9taoY5EywTBMznivistMD3RQuxyiE0TB92Vbu8%3D&reserved=0
2,Methylobacterium litchii,Methylobacterium sp.,is SD21T (=GDMCC 1.4327T=KCTC 8300T,PP600109,SD21,Gao et al. 2025,409.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
3,Methylobacterium litchii,Methylobacterium sp. SD21,is SD21T (=GDMCC 1.4327T=KCTC 8300T,JBBXMY000000000,SD21,Gao et al. 2025,3138810.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
4,Methylobacterium guangdongense,Methylobacterium sp.,is SI9T (=GDMCC 1.14329T= KCTC 8298T,PP600110,SI9,Gao et al. 2025,409.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C60000%7C%7C%7C&sdata=EESf0RJ1F9zstzVMMd%2BGUCGXgkZBUotBItdAa2k1%2BA8%3D&reserved=0
5,Methylobacterium guangdongense,Methylobacterium sp. SI9,is SI9T (=GDMCC 1.14329T= KCTC 8298T,JBBXMX000000000,SI9,Gao et al. 2025,3138811.000000,https://doi.org/10.1099/ijsem.0.006639,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006639%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C7d22bd5abc544b8c9a8308dd3167fac7%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638721045018647513%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiO

### write output to excel

In [24]:
new_df.to_excel(output, engine='xlsxwriter', index = False, na_rep = '') 

### taxonerd test -- skip this

In [ ]:
from taxonerd import TaxoNERD
taxonerd = TaxoNERD(prefer_gpu=False)
nlp = taxonerd.load(model="en_ner_eco_md", exclude=[], linker="taxref", threshold=0.7)
nlp.pipe_names

### option to use biopython to retrieve orgnames based on accessions -- this approarch works but using srcchk for now so this isn't needed

In [ ]:
import Bio
from Bio import Entrez
from Bio import SeqIO

In [ ]:
handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
x = SeqIO.read(handle, 'genbank')
x.annotations['organism']

### extra stuff not using, just saving for now

In [ ]:
combine_df['boo'] = combine_df['PublishedName'] != combine_df['NCBIname']

combine_df.style.applymap('background-color: red', subset=['PublishedName'])

combine_df

In [ ]:
 def color_positive_green(val):
    if combine_df.iloc['boo'] == True:
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color
 
combine_df.style.applymap(color_positive_green)

In [ ]:
#combine_df.apply_style_by_indexes(combine_df[combine_df['PublishedName'] != combine_df['NCBIname']], styler_obj=Styler(bg_color='red'), cols_to_style=['PublishedName', 'NCBIname'])

In [ ]:
#df.style.apply(color, axis=None).to_excel('styled.xlsx', engine='openpyxl')

### Selenium web scraper - title and author note (footnote 1), some of these functions can be turned off following additional testing. Currently not using this function

In [ ]:
#Function to scrape headlines using Selenium
def scrape_with_selenium(url, counter, doi):
    global title
    global footnote
    global description
    title = []
    description = None
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)

    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        #print(title)
    #for element in driver.find_elements(By.CLASS_NAME, "author-footnote-below-abstract"):get.text() #retrieves all footnotes
    for element in driver.find_elements(By.ID, "FN1"): #retrieves just footnote 1
        #print(element.text)
        footnote = element.text
        #print(footnote)
        #print(type(footnote))
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        print(doi)
    
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        print(description)
        if "Description of" in description: 
            print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                #print(description)
        
    #Close the browser window
    driver.quit()
    return title, footnote, description, doi


### Beautifulsoup webscraper - returns abstract, can be turned off following additional testing. 

In [ ]:
def scrape_with_beautifulsoup(url):
    global abstract_text
    abstract_text = None
    response = requests.get(url)
    #soup = BeautifulSoup(response.content, "html.parser")
    soup = BeautifulSoup(response.text, "lxml")
    #print(soup.prettify())
    abstract = soup.find(class_= "articleabstract")
    if abstract is not None:
    #print(abstract.get_text())
        abstract_text = abstract.get_text()
        return abstract_text
    else:
        return
    